# Track targets with TemplateMatch

Targets defined on the first image are tracked on the following images of a ImageDS by using the TemplateMatch function, that implements a template matching based on the orientation correlation algorithm.


In [8]:
import numpy as np

from pathlib import Path

from icepy4d.classes import ImageDS, Targets
from icepy4d.utils.track_targets import TrackTargets

root_dir = Path.cwd().parents[1]

Let's first define some processing parameters

In [9]:
# Define some paths
OUT_DIR = "./results"

TARGETS_DIR = root_dir / "data/targets"
TARGETS_IMAGES_FNAMES = ["IMG_2637.csv", "IMG_1112.csv"]
TARGETS_WORLD_FNAME = "target_world.csv"

IM_DIR = root_dir / "data/img"
cams = ["p1", "p2"]
targets_to_track = ["F2", "F13"]

# Template matching parameters
patch_width = 512
template_width = 16
search_width = 64

debug_viz = False
verbose = True


In [10]:
target_dir = Path(TARGETS_DIR)
targets_image_paths = [target_dir / fname for fname in TARGETS_IMAGES_FNAMES]
targets = Targets(
    im_file_path=targets_image_paths,
    obj_file_path=target_dir / TARGETS_WORLD_FNAME,
)

In [11]:
for cam_id, cam in enumerate(cams):
    print(f"Processing camera {cam}")

    images = ImageDS(Path(IM_DIR) / cam)

    # Define nx2 array with image coordinates of the targets to track
    # in the form of:
    # [x1, y1],
    # [x2, y2]...
    # You can create it manually or use Target class
    targets_coord = np.zeros((len(targets_to_track), 2))
    for i, target in enumerate(targets_to_track):
        targets_coord[i] = targets.get_image_coor_by_label([target], cam_id)[
            0
        ].squeeze()

    # Define TrackTargets object and run tracking
    tracking = TrackTargets(
        images=images,
        patch_centers=targets_coord,
        out_dir=OUT_DIR,
        target_names=targets_to_track,
        patch_width=patch_width,
        template_width=template_width,
        search_width=search_width,
        debug_viz=debug_viz,
        verbose=verbose,
    )
    # tracking.viz_template()
    tracking.track_all_targets()

    print("Done.")

Processing camera p1
Tracking target F2
	Epoch 0 - image: IMG_2637.jpg...  du: 0.24 dv: 0.19 - Coor peak: 256.00 - SNR 7.81
	Epoch 1 - image: IMG_2639.jpg...  du: 0.83 dv: -0.33 - Coor peak: 195.46 - SNR 5.94
	Epoch 2 - image: IMG_2643.jpg...  du: -0.41 dv: 2.74 - Coor peak: 201.19 - SNR 5.85
	Epoch 3 - image: IMG_2650.jpg...  du: 0.61 dv: 0.51 - Coor peak: 179.30 - SNR 5.60
	Epoch 4 - image: IMG_2654.jpg...  du: 0.94 dv: -0.12 - Coor peak: 193.68 - SNR 6.06
	Epoch 5 - image: IMG_2655.jpg...  du: 1.21 dv: -0.78 - Coor peak: 217.68 - SNR 6.87
	Epoch 6 - image: IMG_2658.jpg...  du: 1.72 dv: -2.31 - Coor peak: 218.67 - SNR 6.87
	Epoch 7 - image: IMG_2659.jpg...  du: 1.58 dv: -1.53 - Coor peak: 202.44 - SNR 6.51
	Epoch 8 - image: IMG_2661.jpg...  du: 1.79 dv: -1.84 - Coor peak: 203.81 - SNR 6.50
	Epoch 9 - image: IMG_2663.jpg...  du: 2.41 dv: 0.25 - Coor peak: 202.97 - SNR 6.52
	Epoch 10 - image: IMG_2665.jpg...  du: 1.85 dv: -1.30 - Coor peak: 211.46 - SNR 6.47
	Epoch 11 - image: IMG_2669

In [13]:
tracking.patch_centers

array([[1003.5037, 3859.1558],
       [1565.0667, 3927.6331]])